In [ ]:
import numpy as np
import numpy.random as rd
import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt

# Deep Learning Tutorial Part III: RNNs

Here we implement a **recurrent neural network (RNN)** model for solving a two-alternative forced choice task (2AFC) task, much like the task that Carlos described today. 

We start by defining the model, which has a form similar to the deep network discussed in part 2: 
\begin{gather}
    \mathbf{h}^{(1)} = \phi\left( \mathbf{W}^{rec} \mathbf{h}^{(0)} + \mathbf{W}^{in}\mathbf{x}^{(1)} \right) \\
    \mathbf{h}^{(2)} = \phi\left( \mathbf{W}^{rec} \mathbf{h}^{(1)} + \mathbf{W}^{in}\mathbf{x}^{(2)} \right) \\
    \vdots \\
    \mathbf{h}^{(T)} = \phi\left( \mathbf{W}^{rec} \mathbf{h}^{(T-1)} + \mathbf{W}^{in}\mathbf{x}^{(T)} \right) \\
    \mathbf{y} = \mathbf{W}^{out} \mathbf{h}^{(T)}
\end{gather}
Note the following fundamental differences:
* The weights between each pair of layers are the given by the same parameter $\mathbf{W}^{rec}$
* The input data now consists of a sequence of inputs, so that each layer gets a different input

Since the same weights are shared across layers, we can thus think of the activity across layers $\mathbf{h}^{(t)}, \mathbf{h}^{(t+1)}, \mathbf{h}^{(t+2)}, ...$ as the activity of the same set of neurons at a sequence of times $t, t+1, t+2, ...$, receiving a temporal sequence of inputs $\mathbf{x}^{(t)}, \mathbf{x}^{(t+1)}, \mathbf{x}^{(t+2)}, ...$.

In the task used below, we're not going to do classification, so the output does not have to be a probability and we can dispense of the sigmoid or softmax functions used earlier. Here is the PyTorch code for building this model:

In [ ]:
class RNN(nn.Module):
    def __init__(self, n_neurons):
        super(RNN, self).__init__()
        self.n_neurons = n_neurons # number of neurons
        self.inp = nn.Linear(1, n_neurons) # input weights
        self.Wrec = nn.Parameter(torch.randn(n_neurons, n_neurons) / np.sqrt(n_neurons)) # recurrent weights
        self.out = nn.Linear(n_neurons, 1) # output weights

    def forward(self, inputs):
        """
        Run the RNN with input timecourses
        """
        # Initialize network state
        hidden = torch.zeros(inputs.size(0), self.n_neurons)
        # Run the input through the network
        for i in range(inputs.size(1)):
            output, hidden = self.step(inputs[:, i].unsqueeze(-1), hidden)
        return output.squeeze(), hidden # only output final output and internal state
    
    def step(self, input_ext, hidden):
        hidden = torch.relu(torch.matmul(self.Wrec, hidden.unsqueeze(-1)).squeeze() + self.inp(input_ext))
        output = self.out(hidden)
        return output, hidden

We're going to use SGD to learn the parameters of this model so that it can perform a two-alternative forced choice task consisting of distinguishing between two stimuli:
* class +1: noisy one-dimensional signal with mean of 20.0 (e.g. "amount of sounds" coming from the right, cf. Carlos' click task)
* class -1: noisy one-dimensional signal with mean of -20.0 (e.g. "amount of sounds" coming from the left)

Code to generate these stimuli is provided in the next cell. Below that is code for plotting a few random stimuli to visualize them.

In [ ]:
# Function to generate stimulus
stimulus_mean = 20
stimulus_noise = 0.5
stimulus_duration = 200
def sample_stimuli(batch_size):
    target = torch.sign(torch.randn(batch_size)) # -1 or 1
    stim = target.unsqueeze(-1) * stimulus_mean + stimulus_noise * torch.randn(batch_size, stimulus_duration)
    return target, stim

In [ ]:
# Sample several stimuli and plot them
fig, ax = plt.subplots()
targets, stim = sample_stimuli(5)
for t, s in zip(targets, stim):
    ax.plot(s.numpy().T, label='class %i' % t)
ax.legend()

The job of the network is then to spit out and output of 1.0 if the stimulus is of class +1, and an output of -1.0 if the stimulus is class -1. Note that, even though the network is implicitly classifying these stimuli via its output, it is not classification in the machine learning sense since the network does not output a probability of the stimulus being of one class or the other. Thus, we'll have to use a different loss function. Instead of negative log probability, we'll use the squared difference between the model output and the target output (1.0 or -1.0). 

Below is the code for training the model on this task. Have a look at the code and try running it. 

In [ ]:
# Set up model
n_neurons = 16
model = RNN(n_neurons)

# Set up SGD parameters
n_iter = 500 # iterations of SGD
batch_size = 10
learning_rate = 1e-5
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Squared error loss function
loss_fn = nn.MSELoss()

In [ ]:
# Placeholders
track_loss = np.zeros(n_iter)
track_targets = np.zeros(n_iter * batch_size)
track_outputs = np.zeros(n_iter * batch_size)

# Loop over iterations
for i in range(n_iter):
    if (i + 1) % 50 == 0: # print progress every 100 iterations
        print('%.2f%% iterations of SGD completed...loss = %.2f' % (100* (i + 1) / n_iter, loss))
    # Sample stimulus
    target, stim = sample_stimuli(batch_size)
    # Run model
    outputs, hidden = model(stim)
    # Compute loss
    loss = loss_fn(outputs, target)
    # Keep track of outputs and loss
    track_loss[i] = loss.item()
    track_targets[(i * batch_size) : ((i+1) * batch_size)] = target.detach().numpy()
    track_outputs[(i * batch_size) : ((i+1) * batch_size)] = outputs.detach().numpy()
    # Compute gradients
    optimizer.zero_grad()
    loss.backward()
    # Update weights
    optimizer.step()

In [ ]:
# Plot loss
fig, ax = plt.subplots()
ax.plot(track_loss)
ax.set_yscale('log')

In [ ]:
# Plot model outputs for each stimulus class
fig, ax = plt.subplots()
n_trials_plot = 2000
learned_outputs = track_outputs[-n_trials_plot:]
stim1_responses = learned_outputs[track_targets[-n_trials_plot:] == 1]
stim2_responses = learned_outputs[track_targets[-n_trials_plot:] == -1]
h = ax.hist([stim1_responses, stim2_responses], )
_ = ax.legend(['stimulus class +1', 'stimulus class -1'], frameon=False)
_ = ax.set_xlabel('network output')
_ = ax.set_ylabel('number of iterations')
_ = ax.set_title('last %i iterations only' % n_trials_plot)

### 3.1) How does learning depend on the different task parameters? 

Change the task parameters (i.e. stimulus means, stimulus noise, stimulus duration) and plot the resulting error curves over training.
* Which task parameters makes the task harder or easier for the network to learn? 
* Does learning just get slower when the task gets harder? Or, does learning converge equally quickly but to a much higher error?
* Add a third or fourth stimulus class, along with a target output for each one (e.g. +2, +1, -1, -2). Can you train the network to solve this? If not, try changing the target output to a 2-dimensional vector instead of a scalar for each class.

### 3.2) Change the task to a classification problem

Currently, we are training the network to produce particular outputs for each stimulus class. A similar task would be to ask the network to tell us which class it thinks the stimulus belongs to. Change the output of the network to be a probability distribution over classes (in the case of two classes, this would be a single probability, in the case of more than two this would be vector with of probabilities, one for each class). Then train the network to minimize a classification loss (instead of a squared error). 


### 3.3) Change the model to implement standard biological recurrent network dynamics

The above model implements the machine learning flavor of recurrent neural networks, derived as discrete time dynamical systems. Biological recurrent networks are continuous time dynamical systems. Modify the model to implement a discrete time approximation of a continuous time recurrent network (i.e. Euler integration of a system of ordinary differential equations).

If you don't understand what I'm talking about, see equations 1-9 in [this paper](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1004792).

### 3.4) Change the model to implement standard biological recurrent network dynamics

Add some further biological plausibility to this model by constraining the parameters of the recurrent weight matrix:
* Enforce some level of sparsity by constraining a subset of the weights to stay at 0. This can be done by element-wise multiplying $\mathbf{W}^{rec}$ with a fixed matrix of ones and zeros.
* Enforce Dale's law by constraining the columns of $\mathbf{W}^{rec}$ to be purely negative or purely positive. This can be done by rectifying $\mathbf{W}^{rec}$ (i.e. using `torch.relu()`) and multiplying it from the left by a diagonal matrix with positive and negative ones, specifying which columns to be excitatory and inhibitory, respectively.

If you're having trouble, see the above linked paper for further details.